## VRP(Vehicle Routing Problem)
* 车辆路径问题，可以看成旅行商问题的推广
* 有N辆车，都从原点出发，每辆车访问一些点后回到原点，要求所有 的点都要被访问到，求最短的车辆行驶距离或最少需要的车辆数

In [1]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
 

# 设置城市名
city_names = ["New York", "Los Angeles", "Chicago", "Minneapolis", "Denver", "Dallas", "Seattle",
            "Boston", "San Francisco", "St. Louis", "Houston", "Phoenix", "Salt Lake City"]    

# 求两个城市之间的距离，可以采用地图工具API 得出
# 设置数据
def create_data_model():
    data = {}
    # 设置3个维度： 邻接矩阵（城市之间距离），车的数量，起始点
    data['distance_matrix'] = [
        [   0, 2451,  713, 1018, 1631, 1374, 2408,  213, 2571,  875, 1420, 2145, 1972], # New York
        [2451,    0, 1745, 1524,  831, 1240,  959, 2596,  403, 1589, 1374,  357,  579], # Los Angeles
        [ 713, 1745,    0,  355,  920,  803, 1737,  851, 1858,  262,  940, 1453, 1260], # Chicago
        [1018, 1524,  355,    0,  700,  862, 1395, 1123, 1584,  466, 1056, 1280,  987], # Minneapolis
        [1631,  831,  920,  700,    0,  663, 1021, 1769,  949,  796,  879,  586,  371], # Denver
        [1374, 1240,  803,  862,  663,    0, 1681, 1551, 1765,  547,  225,  887,  999], # Dallas
        [2408,  959, 1737, 1395, 1021, 1681,    0, 2493,  678, 1724, 1891, 1114,  701], # Seattle
        [ 213, 2596,  851, 1123, 1769, 1551, 2493,    0, 2699, 1038, 1605, 2300, 2099], # Boston
        [2571,  403, 1858, 1584,  949, 1765,  678, 2699,    0, 1744, 1645,  653,  600], # San Francisco
        [ 875, 1589,  262,  466,  796,  547, 1724, 1038, 1744,    0,  679, 1272, 1162], # St. Louis
        [1420, 1374,  940, 1056,  879,  225, 1891, 1605, 1645,  679,    0, 1017, 1200], # Houston
        [2145,  357, 1453, 1280,  586,  887, 1114, 2300,  653, 1272, 1017,    0,  504], # Phoenix
        [1972,  579, 1260,  987,  371,  999,  701, 2099,  600, 1162,  1200,  504,   0]] # Salt Lake City
    data['num_vehicles'] = 1
    data['depot'] = 0 #起始点的ID
    return data
 
# 输出结果
def print_solution(manager, routing, solution):
    print('总行驶里程: {} 公里'.format(solution.ObjectiveValue()))
    # 因为depot=0，所以Start(0)
    index = routing.Start(0)
    plan_output = '车辆的路径:\n'
    route_distance = 0
    # 判断是否达到终点
    while not routing.IsEnd(index):
        # 显示输出的路径规划
        plan_output += ' {} ->'.format(city_names[manager.IndexToNode(index)])
        previous_index = index
        # 得到下一个城市
        index = solution.Value(routing.NextVar(index))
        # 计算起点到终点的距离 => 添加到总距离中
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    # 输出最后一个城市
    plan_output += city_names[manager.IndexToNode(index)]
    print(plan_output)
    print('{}公里'.format(route_distance))

def main():
    # 初始化数据
    data = create_data_model()
    ###################step1#######################
    # 创建路线管理，tsp_size (邻接矩阵表的大小), num_vehicles（车的数量）, depot（原点）
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])
 
    # 创建路径规划模型 Routing Model.
    routing = pywrapcp.RoutingModel(manager)
    
    # 计算两点之间的距离
    def distance_callback(from_index, to_index):
        # 从Index => NodeIndex的转换
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        # 从distance_matrix中的起点、终点 找到距离
        return data['distance_matrix'][from_node][to_node]
 
    # 创建距离回调函数，用于计算两个节点之间的距离
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    # 对所有的车辆，采用相同的distance callback的计算
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
    ###################step2######################
    # 设置启发式方法（距离成本最短 PATH_CHEAPEST_ARC）
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    # 初始化的解,速度快即可
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    # 在初始解的基础上，进一步优化（使用local_search,通过邻居调整）
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    #搜索时间限制 1s
    search_parameters.time_limit.seconds = 1

    #print(routing)
    # 求解路径规划
    solution = routing.SolveWithParameters(search_parameters)
    '''
        solution=0 => ROUTING_NOT_SOLVED:问题还没有解决。
        solution=1 => ROUTING_SUCCESS:圆满地解决了问题。
        solution=2 => ROUTING_FAIL:没有找到解决这个问题的办法。
        solution=3 => ROUTING_FAIL_TIMEOUT:找到解决办法前的时间限制。
        solution=4 => ROUTING_INVALID:模型、模型参数或标志无效。  
    '''
    # 输出结果
    if solution:
        print_solution(manager, routing, solution)

if __name__ == '__main__':
    main()
# 7293 ，只用了初始方法
# 7569 ，

总行驶里程: 7293 公里
车辆的路径:
 New York -> Boston -> Chicago -> Minneapolis -> Denver -> Salt Lake City -> Seattle -> San Francisco -> Los Angeles -> Phoenix -> Houston -> Dallas -> St. Louis ->New York
7293公里


In [2]:
help(routing_enums_pb2.FirstSolutionStrategy)

Help on class FirstSolutionStrategy in module ortools.constraint_solver.routing_enums_pb2:

class FirstSolutionStrategy(google.protobuf.message.Message)
 |  FirstSolutionStrategy(**kwargs)
 |  
 |  Abstract base class for protocol messages.
 |  
 |  Protocol message classes are almost always generated by the protocol
 |  compiler.  These generated types subclass Message and implement the methods
 |  shown below.
 |  
 |  Method resolution order:
 |      FirstSolutionStrategy
 |      google.protobuf.message.Message
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  ByteSize(self)
 |  
 |  Clear = _Clear(self)
 |  
 |  ClearField(self, field_name)
 |  
 |  DiscardUnknownFields = _DiscardUnknownFields(self)
 |  
 |  FindInitializationErrors(self)
 |      Finds required fields which are not initialized.
 |      
 |      Returns:
 |        A list of strings.  Each string is a path to an uninitialized field from
 |        the top-level message, e.g. "foo.bar[5].baz".
 |  
 |  Ha